In [1]:
pwd

'/private/home/jeanremi/project/jrr/code/meg_experiment'

In [2]:
%matplotlib inline

import os.path as op
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
import submitit
import mne
from common import log_files, data_path, read_log, get_log_times

In [3]:
log_files = log_files.query('task=="visual"')

In [4]:
def get_evoked(subject):
    log_file = log_files.iloc[subject]
    # file name
    raw_fname = op.join(data_path, log_file['meg_file'])
    log_fname = op.join(data_path, log_file['log_file'])

    # preprocess meg data
    raw = mne.io.read_raw_ctf(raw_fname, preload=True, verbose=False)
    raw.filter(.1, 40., verbose=False)
    events = mne.find_events(raw, verbose=False)

    # preprocess annotations
    log = read_log(log_fname)

    # link meg and annotations
    log = get_log_times(log, events,  raw.info['sfreq'])

    # Segment into word-locked epochs
    log_events = np.c_[log.meg_sample, np.ones((len(log), 2), int)]
    _, idx = np.unique(log_events[:, 0], return_index=True)
    picks = mne.pick_types(raw.info, meg=True, eeg=False,
                           stim=False, eog=False, ecg=False)
    epochs = mne.Epochs(
        raw,
        events=log_events[idx], metadata=log.iloc[idx],
        tmin=-.100, tmax=1., decim=20,
        picks=picks, preload=True,
        verbose=False
    )
    epochs = epochs['condition=="word"']
    meg = np.median(epochs.get_data(), axis=0)
    
    evo = mne.EvokedArray(meg, epochs.info, tmin=-.100)
    evo = remap(evo)
    
    return evo

In [5]:
def remap(evo):
    from mne import EvokedArray
    from mne.datasets import sample, spm_face
    from mne.channels.interpolation import _map_meg_channels
    from mne import io

    # CTF
    evoked_ctf = evo

    # Elekta
    data_path = sample.data_path() + '/MEG/sample/sample_audvis_filt-0-40_raw'
    raw = mne.io.read_raw_fif(data_path + '.fif')
    events = mne.read_events(data_path + '-eve.fif')
    epochs_elk = mne.Epochs(raw, events, {'AudL': 1, 'AudR': 2}, -0.050, 0.400,
                            preload=True, decim=2, verbose=False)
    epochs_elk.pick_types(meg=True, eeg=False, exclude=list())
    evoked_elk = epochs_elk.average()

    # Map
    mapping = _map_meg_channels(evoked_ctf.info, evoked_elk.info, mode='accurate')
    new = mapping.dot(evoked_ctf.data)
    evoked_new = EvokedArray(new, evoked_elk.info, evoked_ctf.times[0])
    return evoked_new

In [ ]:
log_path = op.join(data_path, '..', 'jrr', 'log')
executor = submitit.AutoExecutor(folder=log_path)
executor.update_parameters(timeout_min=10, 
                           partition='learnfair,uninterrupted,scavenge',
                           constraint='pascal', 
                           cpus_per_task=4)

jobs = list()
folder = 'jrr_iclr'
for subject in trange(len(log_files)):
    jobs.append([subject,])
jobs = executor.map_array(get_evoked, *zip(*jobs))

In [ ]:
# retrieve results
all_evo = list()
for job in jobs:
    print('*', end='')
    try:
        evo = job.result()
        all_evo.append(evo.data)
    except:
        pass
evoked = mne.EvokedArray(np.mean(all_evo, 0), evo.info,  tmin=evo.times[0])

In [ ]:
evoked = get_evoked(0)

In [ ]:
evoked = mne.EvokedArray(np.mean(all_evo, 0), evo.info,  tmin=evo.times[0])

In [ ]:
fig = plt.figure()
axes = [fig.add_subplot(2, 4 , idx+1) for idx in range(4)]
ax = fig.add_subplot(2, 1 , 2)

tois = [0., .075, .125, .350]
evoked.plot_topomap(times=tois,
                   ch_type='grad', sensors=False, axes=axes,
                   colorbar=False, show=False, time_unit='ms');
evoked.plot(spatial_colors=True, 
            picks=mne.pick_types(evoked.info, 'mag'), axes=[ax],
            show=False, time_unit='ms', window_title=False);
for t in tois:
    ax.axvline(t*1e3, color='k', lw=.5)
ax.set_title('')
ax.set_xlim(-100, 600)
ax.set_xlabel('Time from word onset (ms)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

fig.savefig('meg_evoked.pdf')

In [ ]:
from scipy.stats import pearsonr
r = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        r[i, j], _ = pearsonr(features[:,i], features[:,j])

In [ ]:
plt.matshow(r, cmap='RdBu_r', vmin=-1, vmax=1)
plt.colorbar()